In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle as pkl



import seaborn as sns
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

from IPython.display import display, HTML

import ipywidgets as widgets
from ipywidgets import Layout

from difflib import get_close_matches
from scipy.signal import savgol_filter

import h5py
from joblib import Parallel, delayed

import random
random.seed(22)

In [2]:
sys.path.append("../code/")
from extract_angles import get_bend_angles, get_tan_angles
from extract_curvatures import get_curv_savgol

In [3]:
%matplotlib widget

## Get the skeletons per each class

In [4]:
skel_file_folder_test = "/share/data/temp/athira/tierpsy_skeleton_files_lightON_npy/"
skel_file_folder_wild = "/share/data/temp/athira/tierpsy_skeleton_files_for_eigen_npy/"
#skel_file_folder = "/media/athira/Seagate Expansion Drive/Backup_Feb26_2020/athira_data/tierpsy_skeleton_files_for_eigen_npy/"

In [5]:
# From the saved skeleton arrays, pick the ones belonging to test group 

skel_files_test = [os.path.join(root,name) for root,dirs,filenames in os.walk(skel_file_folder_test) 
                                              for name in filenames if ((name.endswith('.npy')) & 
                                                                        (name.split("_")[5] == "None") &
                                                                        (name.split("_")[7] == "Light")
                                                                       )]
#print([os.path.basename(fname).split("_")[4] for fname in skel_files_control])
print(f"Number of skeleton files which belong to control group:{len(skel_files_test)}")

Number of skeleton files which belong to control group:99


In [6]:
# From the saved skeleton arrays, pick the ones belonging to test group 

skel_files_wild = [os.path.join(root,name) for root,dirs,filenames in os.walk(skel_file_folder_wild) 
                                              for name in filenames if ((name.endswith('.npy')) & 
                                                                        (name.split("_")[5] == "None") &
                                                                        (name.split("_")[7] == "None")
                                                                       )]
#print([os.path.basename(fname).split("_")[4] for fname in skel_files_control])
print(f"Number of skeleton files which belong to control group:{len(skel_files_wild)}")

Number of skeleton files which belong to control group:87


### Calculate features

In [7]:
# Load all the skeletons as a list of 3d arrays

skel_arrays_test = [np.load(skel_file) for skel_file in skel_files_test]
skel_arrays_wild = [np.load(skel_file) for skel_file in skel_files_wild]

In [8]:
tail_end = -1
skel_arrays_test = [skel_arr[:,:tail_end,:] for skel_arr in skel_arrays_test]
skel_arrays_wild = [skel_arr[:,:tail_end,:] for skel_arr in skel_arrays_wild]

In [9]:
def get_curv_list(skel_arrays):
    
    curv_list = Parallel(n_jobs= 10, verbose = 5)(delayed(get_curv_savgol)(skel_arr) 
                                                for skel_arr in skel_arrays)
    return curv_list

In [10]:

curv_list_test = get_curv_list(skel_arrays_test)
curv_matrix_test = np.vstack(curv_list_test)

curv_list_wild = get_curv_list(skel_arrays_wild)
curv_matrix_wild = np.vstack(curv_list_wild)

[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  54 tasks      | elapsed:    1.1s
[Parallel(n_jobs=10)]: Done  80 out of  99 | elapsed:    1.2s remaining:    0.3s
[Parallel(n_jobs=10)]: Done  99 out of  99 | elapsed:    1.3s finished
[Parallel(n_jobs=10)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=10)]: Done  68 out of  87 | elapsed:    0.5s remaining:    0.1s
[Parallel(n_jobs=10)]: Done  87 out of  87 | elapsed:    0.6s finished


### Get eigen dictionary calculated from wild control type

In [11]:
# 
eig_dict = np.load('eigen_dict_all.npy', allow_pickle=True).item()

In [12]:
### Calculate ects for the test group

eig_vec_matrix = np.array(eig_dict['eig_vecs'][:,0:eig_dict['n_modes']])
eig_vec_matrix.shape

(48, 6)

### Get ects values

In [13]:
ects_test = np.dot(curv_matrix_test,eig_vec_matrix)
ects_test.shape

(463246, 6)

In [14]:
ects_wild = np.dot(curv_matrix_wild,eig_vec_matrix)
ects_wild.shape

(809320, 6)

### Density plots for ects

In [15]:
from scipy.stats import gaussian_kde
  
fig, axes = plt.subplots(eig_dict['n_modes'],1,figsize=(6,12),sharex=True)

for i in range(eig_dict['n_modes']):
        sns.set_style('whitegrid')
        sns.kdeplot(ects_test[:,i], ax=axes[i], label='test', clip=(-1,1))
        sns.kdeplot(ects_wild[:,i], ax=axes[i], label='wild', clip=(-1,1))
        
        axes[i].set_xlabel(f"eigen coeff {i}")
        axes[i].set_ylabel('density')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
from scipy.stats import gaussian_kde
  
fig, axes = plt.subplots(1,1,figsize=(10,5))

for i in range(eig_dict['n_modes']):
        sns.set_style('whitegrid')
        sns.kdeplot(ects_wild[:,i], ax=axes, label=f"coeff_{i}")
        axes.set_xlabel('eigen coefficient value')
        axes.set_ylabel('density')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …